In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input, Flatten, Activation, PReLU
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.activations import linear, relu

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from numpy import sqrt

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

In [3]:
%%time
cat = [f'col_{i}' for i in range(1, 117)]
con = [f'col_{i}' for i in range(117, 131)]
for c in cat:
    train_df[c] = train_df[c].astype('category')
    test_df[c] = test_df[c].astype('category')
    train_df[c] = train_df[c].cat.codes
    test_df[c] = test_df[c].cat.codes
del c
train_df.drop('id', axis=1, inplace=True)
test_df.drop('id', axis=1, inplace=True)

Wall time: 9.76 s


In [4]:
X = train_df.iloc[:,train_df.columns != 'target'].values
y = train_df.iloc[:,train_df.columns == 'target'].values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)
del X, y

In [6]:
activation_fn = 'relu'
model = Sequential()
# Input Layer
model.add(Input(shape=(130,)))

# First Hidden Layer
model.add(Dense(260, activation=activation_fn))
model.add(Dropout(.3))

# Second Hidden Layer
model.add(Dense(520, activation=activation_fn))
model.add(Dropout(.4))

# Third Hidden Layer
model.add(Dense(260, activation=activation_fn))
model.add(Dropout(.3))

# Fourth Hidden Layer
model.add(Dense(130, activation=activation_fn))
model.add(Dropout(.2))

# Firth Hidden Layer
model.add(Dense(65, activation=activation_fn))
model.add(Dropout(.1))

# Sixth Hidden Layer
model.add(Dense(32, activation=activation_fn))

# Output Layer
model.add(Dense(1, activation=linear))
model.compile(optimizer=RMSprop(learning_rate=1e-3), loss=MeanSquaredError(), metrics=['mse', 'mae'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 260)               34060     
_________________________________________________________________
dropout (Dropout)            (None, 260)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 520)               135720    
_________________________________________________________________
dropout_1 (Dropout)          (None, 520)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 260)               135460    
_________________________________________________________________
dropout_2 (Dropout)          (None, 260)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 130)               3

In [7]:
mse_history = []
mae_history = []

In [8]:
%%time
init_epochs = 0
epochs_now = 100
history = model.fit(X_train, y_train, epochs=epochs_now, validation_data=(X_val, y_val), initial_epoch=init_epochs)
mse_history += history.history['mse']
mae_history += history.history['mae']
model.save(f'relu_linear_6/model_{epochs_now}.h5')

Epoch 1/100
2980/2980 [==============================] - 22s 7ms/step - loss: 5956222.5000 - mse: 5956222.5000 - mae: 1531.5386 - val_loss: 5801152.5000 - val_mse: 5801152.5000 - val_mae: 1419.1067
Epoch 2/100
2980/2980 [==============================] - 22s 7ms/step - loss: 5178345.0000 - mse: 5178345.0000 - mae: 1410.7272 - val_loss: 4509687.0000 - val_mse: 4509687.0000 - val_mae: 1269.6991
Epoch 3/100
2980/2980 [==============================] - 28s 9ms/step - loss: 5071676.5000 - mse: 5071676.5000 - mae: 1395.2234 - val_loss: 4114221.7500 - val_mse: 4114221.7500 - val_mae: 1335.0708
Epoch 4/100
2980/2980 [==============================] - 18s 6ms/step - loss: 5040193.5000 - mse: 5040193.5000 - mae: 1388.3733 - val_loss: 4238910.0000 - val_mse: 4238910.0000 - val_mae: 1259.4462
Epoch 5/100
2980/2980 [==============================] - 18s 6ms/step - loss: 4943599.5000 - mse: 4943599.5000 - mae: 1371.8440 - val_loss: 5911537.0000 - val_mse: 5911537.0000 - val_mae: 1786.7345
Epoch 6/10

2980/2980 [==============================] - 19s 6ms/step - loss: 4833700.5000 - mse: 4833700.5000 - mae: 1361.4705 - val_loss: 4026921.0000 - val_mse: 4026921.0000 - val_mae: 1275.2469
Epoch 43/100
2980/2980 [==============================] - 21s 7ms/step - loss: 4740359.0000 - mse: 4740359.0000 - mae: 1358.5602 - val_loss: 4200600.5000 - val_mse: 4200600.5000 - val_mae: 1436.7325
Epoch 44/100
2980/2980 [==============================] - 20s 7ms/step - loss: 4829784.5000 - mse: 4829784.5000 - mae: 1365.8002 - val_loss: 4031304.2500 - val_mse: 4031304.2500 - val_mae: 1326.7053
Epoch 45/100
2980/2980 [==============================] - 23s 8ms/step - loss: 4766520.0000 - mse: 4766520.0000 - mae: 1359.6530 - val_loss: 4055953.7500 - val_mse: 4055953.7500 - val_mae: 1339.5143
Epoch 46/100
2980/2980 [==============================] - 23s 8ms/step - loss: 4774126.5000 - mse: 4774126.5000 - mae: 1355.2256 - val_loss: 4052825.7500 - val_mse: 4052825.7500 - val_mae: 1311.0438
Epoch 47/100
2980/

2980/2980 [==============================] - 306s 103ms/step - loss: 5940214.5000 - mse: 5940214.5000 - mae: 1350.4452 - val_loss: 4276775.0000 - val_mse: 4276775.0000 - val_mae: 1364.1213
Epoch 84/100
2980/2980 [==============================] - 14s 5ms/step - loss: 4771897.5000 - mse: 4771897.5000 - mae: 1356.1959 - val_loss: 4355779.5000 - val_mse: 4355779.5000 - val_mae: 1498.7136
Epoch 85/100
2980/2980 [==============================] - 11s 4ms/step - loss: 4658870.5000 - mse: 4658870.5000 - mae: 1349.7668 - val_loss: 3967283.7500 - val_mse: 3967283.7500 - val_mae: 1267.5707
Epoch 86/100
2980/2980 [==============================] - 10s 3ms/step - loss: 4637816.5000 - mse: 4637816.5000 - mae: 1342.3295 - val_loss: 4165893.5000 - val_mse: 4165893.5000 - val_mae: 1410.5414
Epoch 87/100
2980/2980 [==============================] - 10s 3ms/step - loss: 4649700.5000 - mse: 4649700.5000 - mae: 1343.0328 - val_loss: 4146737.0000 - val_mse: 4146737.0000 - val_mae: 1364.6862
Epoch 88/100
29

In [9]:
%%time
init_epochs = 100
epochs_now = 500
history = model.fit(X_train, y_train, epochs=epochs_now, validation_data=(X_val, y_val), initial_epoch=init_epochs)
mse_history += history.history['mse']
mae_history += history.history['mae']
model.save(f'relu_linear_6/model_{epochs_now}.h5')

Epoch 101/500
2980/2980 [==============================] - 22s 7ms/step - loss: 4701498.5000 - mse: 4701498.5000 - mae: 1342.7797 - val_loss: 3955369.5000 - val_mse: 3955369.5000 - val_mae: 1329.0306
Epoch 102/500
2980/2980 [==============================] - 21s 7ms/step - loss: 4781446.5000 - mse: 4781446.5000 - mae: 1404.9426 - val_loss: 4332983.5000 - val_mse: 4332983.5000 - val_mae: 1398.1360
Epoch 103/500
2980/2980 [==============================] - 22s 7ms/step - loss: 4643235.5000 - mse: 4643235.5000 - mae: 1344.1274 - val_loss: 4104968.7500 - val_mse: 4104968.7500 - val_mae: 1318.6069
Epoch 104/500
2980/2980 [==============================] - 25s 8ms/step - loss: 4724837.5000 - mse: 4724837.5000 - mae: 1347.6094 - val_loss: 4104290.5000 - val_mse: 4104290.5000 - val_mae: 1324.0704
Epoch 105/500
2980/2980 [==============================] - 24s 8ms/step - loss: 4652900.5000 - mse: 4652900.5000 - mae: 1341.8131 - val_loss: 4018978.0000 - val_mse: 4018978.0000 - val_mae: 1259.1400


2980/2980 [==============================] - 20s 7ms/step - loss: 4652869.0000 - mse: 4652869.0000 - mae: 1343.0031 - val_loss: 4085326.7500 - val_mse: 4085326.7500 - val_mae: 1311.0277
Epoch 142/500
2980/2980 [==============================] - 21s 7ms/step - loss: 4698099.0000 - mse: 4698099.0000 - mae: 1346.7251 - val_loss: 4016555.2500 - val_mse: 4016555.2500 - val_mae: 1338.1407
Epoch 143/500
2980/2980 [==============================] - 21s 7ms/step - loss: 4639429.0000 - mse: 4639429.0000 - mae: 1344.4053 - val_loss: 3969064.2500 - val_mse: 3969064.2500 - val_mae: 1302.5658
Epoch 144/500
2980/2980 [==============================] - 23s 8ms/step - loss: 4693966.0000 - mse: 4693966.0000 - mae: 1345.7957 - val_loss: 4040369.5000 - val_mse: 4040369.5000 - val_mae: 1319.6177
Epoch 145/500
2980/2980 [==============================] - 22s 8ms/step - loss: 4802333.0000 - mse: 4802333.0000 - mae: 1351.5746 - val_loss: 3916709.5000 - val_mse: 3916709.5000 - val_mae: 1265.0254
Epoch 146/500


2980/2980 [==============================] - 34s 12ms/step - loss: 4600016.0000 - mse: 4600016.0000 - mae: 1323.2642 - val_loss: 4111853.2500 - val_mse: 4111853.2500 - val_mae: 1378.0294
Epoch 182/500
2980/2980 [==============================] - 34s 11ms/step - loss: 4678407.0000 - mse: 4678407.0000 - mae: 1329.9779 - val_loss: 4628045.0000 - val_mse: 4628045.0000 - val_mae: 1529.5160
Epoch 183/500
2980/2980 [==============================] - 33s 11ms/step - loss: 4746025.0000 - mse: 4746025.0000 - mae: 1333.1799 - val_loss: 4037597.5000 - val_mse: 4037597.5000 - val_mae: 1273.0890
Epoch 184/500
2980/2980 [==============================] - 36s 12ms/step - loss: 4680424.0000 - mse: 4680424.0000 - mae: 1331.1559 - val_loss: 4372713.5000 - val_mse: 4372713.5000 - val_mae: 1393.2472
Epoch 185/500
2980/2980 [==============================] - 36s 12ms/step - loss: 4698919.5000 - mse: 4698919.5000 - mae: 1338.5112 - val_loss: 3960436.5000 - val_mse: 3960436.5000 - val_mae: 1349.3264
Epoch 186

2980/2980 [==============================] - 36s 12ms/step - loss: 4619142.0000 - mse: 4619142.0000 - mae: 1331.1248 - val_loss: 4066522.7500 - val_mse: 4066522.7500 - val_mae: 1391.5961
Epoch 222/500
2980/2980 [==============================] - 34s 11ms/step - loss: 4630163.5000 - mse: 4630163.5000 - mae: 1330.4860 - val_loss: 4168718.7500 - val_mse: 4168718.7500 - val_mae: 1448.1073
Epoch 223/500
2980/2980 [==============================] - 33s 11ms/step - loss: 4546380.5000 - mse: 4546380.5000 - mae: 1324.7703 - val_loss: 4647570.5000 - val_mse: 4647570.5000 - val_mae: 1434.6143
Epoch 224/500
2980/2980 [==============================] - 27s 9ms/step - loss: 4578137.5000 - mse: 4578137.5000 - mae: 1324.0237 - val_loss: 4149793.2500 - val_mse: 4149793.2500 - val_mae: 1400.4607
Epoch 225/500
2980/2980 [==============================] - 21s 7ms/step - loss: 4466644.5000 - mse: 4466644.5000 - mae: 1315.5259 - val_loss: 3970153.5000 - val_mse: 3970153.5000 - val_mae: 1297.1774
Epoch 226/5

2980/2980 [==============================] - 12s 4ms/step - loss: 4782247.0000 - mse: 4782247.0000 - mae: 1326.8251 - val_loss: 4149272.0000 - val_mse: 4149272.0000 - val_mae: 1395.5259
Epoch 263/500
2980/2980 [==============================] - 15s 5ms/step - loss: 4562437.0000 - mse: 4562437.0000 - mae: 1319.2473 - val_loss: 3851167.0000 - val_mse: 3851167.0000 - val_mae: 1260.5266
Epoch 264/500
2980/2980 [==============================] - 16s 6ms/step - loss: 4486646.0000 - mse: 4486646.0000 - mae: 1314.2915 - val_loss: 4032420.7500 - val_mse: 4032420.7500 - val_mae: 1374.9286
Epoch 265/500
2980/2980 [==============================] - 17s 6ms/step - loss: 4525304.0000 - mse: 4525304.0000 - mae: 1318.0226 - val_loss: 4084455.2500 - val_mse: 4084455.2500 - val_mae: 1348.9384
Epoch 266/500
2980/2980 [==============================] - 15s 5ms/step - loss: 4493318.0000 - mse: 4493318.0000 - mae: 1312.7352 - val_loss: 4117755.7500 - val_mse: 4117755.7500 - val_mae: 1327.0957
Epoch 267/500


2980/2980 [==============================] - 15s 5ms/step - loss: 4515585.5000 - mse: 4515585.5000 - mae: 1319.7043 - val_loss: 4404860.5000 - val_mse: 4404860.5000 - val_mae: 1440.1738
Epoch 304/500
2980/2980 [==============================] - 15s 5ms/step - loss: 4537615.0000 - mse: 4537615.0000 - mae: 1322.7799 - val_loss: 4052363.0000 - val_mse: 4052363.0000 - val_mae: 1226.6365
Epoch 305/500
2980/2980 [==============================] - 15s 5ms/step - loss: 4531793.5000 - mse: 4531793.5000 - mae: 1321.7413 - val_loss: 4041372.5000 - val_mse: 4041372.5000 - val_mae: 1269.1837
Epoch 306/500
2980/2980 [==============================] - 15s 5ms/step - loss: 4584358.0000 - mse: 4584358.0000 - mae: 1324.3350 - val_loss: 3827341.5000 - val_mse: 3827341.5000 - val_mae: 1273.3010
Epoch 307/500
2980/2980 [==============================] - 14s 5ms/step - loss: 4590827.0000 - mse: 4590827.0000 - mae: 1327.4352 - val_loss: 3979240.2500 - val_mse: 3979240.2500 - val_mae: 1342.0637
Epoch 308/500


2980/2980 [==============================] - 14s 5ms/step - loss: 4495912.5000 - mse: 4495912.5000 - mae: 1323.2546 - val_loss: 3983583.2500 - val_mse: 3983583.2500 - val_mae: 1373.8738
Epoch 345/500
2980/2980 [==============================] - 14s 5ms/step - loss: 4512060.0000 - mse: 4512060.0000 - mae: 1320.8715 - val_loss: 4285015.0000 - val_mse: 4285015.0000 - val_mae: 1427.5980
Epoch 346/500
2980/2980 [==============================] - 14s 5ms/step - loss: 4536095.5000 - mse: 4536095.5000 - mae: 1321.0914 - val_loss: 4286504.0000 - val_mse: 4286504.0000 - val_mae: 1475.1167
Epoch 347/500
2980/2980 [==============================] - 14s 5ms/step - loss: 4620657.5000 - mse: 4620657.5000 - mae: 1324.6017 - val_loss: 3953339.5000 - val_mse: 3953339.5000 - val_mae: 1305.2070
Epoch 348/500
2980/2980 [==============================] - 14s 5ms/step - loss: 4642367.0000 - mse: 4642367.0000 - mae: 1320.9886 - val_loss: 4010512.2500 - val_mse: 4010512.2500 - val_mae: 1354.9127
Epoch 349/500


2980/2980 [==============================] - 11s 4ms/step - loss: 4538363.0000 - mse: 4538363.0000 - mae: 1311.9497 - val_loss: 3885585.5000 - val_mse: 3885585.5000 - val_mae: 1241.6149
Epoch 386/500
2980/2980 [==============================] - 16s 5ms/step - loss: 4610051.0000 - mse: 4610051.0000 - mae: 1315.6687 - val_loss: 3924143.2500 - val_mse: 3924143.2500 - val_mae: 1306.7036
Epoch 387/500
2980/2980 [==============================] - 14s 5ms/step - loss: 4546951.5000 - mse: 4546951.5000 - mae: 1317.1270 - val_loss: 4014417.5000 - val_mse: 4014417.5000 - val_mae: 1317.5850
Epoch 388/500
2980/2980 [==============================] - 15s 5ms/step - loss: 4518412.0000 - mse: 4518412.0000 - mae: 1313.4918 - val_loss: 3874393.7500 - val_mse: 3874393.7500 - val_mae: 1245.5769
Epoch 389/500
2980/2980 [==============================] - 16s 5ms/step - loss: 4528496.5000 - mse: 4528496.5000 - mae: 1308.1045 - val_loss: 3929098.2500 - val_mse: 3929098.2500 - val_mae: 1300.1714
Epoch 390/500


2980/2980 [==============================] - 13s 4ms/step - loss: 4589642.5000 - mse: 4589642.5000 - mae: 1322.0348 - val_loss: 4117450.2500 - val_mse: 4117450.2500 - val_mae: 1363.0370
Epoch 427/500
2980/2980 [==============================] - 13s 4ms/step - loss: 4686949.5000 - mse: 4686949.5000 - mae: 1320.3242 - val_loss: 3868394.5000 - val_mse: 3868394.5000 - val_mae: 1233.5417
Epoch 428/500
2980/2980 [==============================] - 12s 4ms/step - loss: 4534674.5000 - mse: 4534674.5000 - mae: 1316.1648 - val_loss: 3912366.5000 - val_mse: 3912366.5000 - val_mae: 1287.1387
Epoch 429/500
2980/2980 [==============================] - 17s 6ms/step - loss: 4817962.0000 - mse: 4817962.0000 - mae: 1314.4518 - val_loss: 3894019.0000 - val_mse: 3894019.0000 - val_mae: 1273.8073
Epoch 430/500
2980/2980 [==============================] - 16s 6ms/step - loss: 4622127.5000 - mse: 4622127.5000 - mae: 1317.0443 - val_loss: 3957937.5000 - val_mse: 3957937.5000 - val_mae: 1308.9440
Epoch 431/500


2980/2980 [==============================] - 11s 4ms/step - loss: 4576945.5000 - mse: 4576945.5000 - mae: 1327.0809 - val_loss: 3974069.5000 - val_mse: 3974069.5000 - val_mae: 1315.6771
Epoch 468/500
2980/2980 [==============================] - 9s 3ms/step - loss: 4616231.0000 - mse: 4616231.0000 - mae: 1327.8425 - val_loss: 3972751.7500 - val_mse: 3972751.7500 - val_mae: 1320.0159
Epoch 469/500
2980/2980 [==============================] - 9s 3ms/step - loss: 4623223.0000 - mse: 4623223.0000 - mae: 1329.0419 - val_loss: 4178417.5000 - val_mse: 4178417.5000 - val_mae: 1425.3298
Epoch 470/500
2980/2980 [==============================] - 9s 3ms/step - loss: 4668721.0000 - mse: 4668721.0000 - mae: 1332.6516 - val_loss: 3979617.5000 - val_mse: 3979617.5000 - val_mae: 1342.5706
Epoch 471/500
2980/2980 [==============================] - 12s 4ms/step - loss: 4653371.0000 - mse: 4653371.0000 - mae: 1333.0021 - val_loss: 4097146.0000 - val_mse: 4097146.0000 - val_mae: 1411.8525
Epoch 472/500
298

In [10]:
%%time
init_epochs = 500
epochs_now = 1000
history = model.fit(X_train, y_train, epochs=epochs_now, validation_data=(X_val, y_val), initial_epoch=init_epochs)
mse_history += history.history['mse']
mae_history += history.history['mae']
model.save(f'relu_linear_6/model_{epochs_now}.h5')

Epoch 501/1000
2980/2980 [==============================] - 10s 3ms/step - loss: 4665988.0000 - mse: 4665988.0000 - mae: 1336.2806 - val_loss: 3963781.2500 - val_mse: 3963781.2500 - val_mae: 1284.8362
Epoch 502/1000
2980/2980 [==============================] - 10s 3ms/step - loss: 5137765.0000 - mse: 5137765.0000 - mae: 1337.4628 - val_loss: 4227962.0000 - val_mse: 4227962.0000 - val_mae: 1447.6891
Epoch 503/1000
2980/2980 [==============================] - 10s 3ms/step - loss: 4604927.0000 - mse: 4604927.0000 - mae: 1330.6434 - val_loss: 4020679.2500 - val_mse: 4020679.2500 - val_mae: 1245.9572
Epoch 504/1000
2980/2980 [==============================] - 10s 3ms/step - loss: 4726160.0000 - mse: 4726160.0000 - mae: 1334.5209 - val_loss: 4261427.0000 - val_mse: 4261427.0000 - val_mae: 1449.3920
Epoch 505/1000
2980/2980 [==============================] - 13s 4ms/step - loss: 4641180.5000 - mse: 4641180.5000 - mae: 1334.3088 - val_loss: 4220048.0000 - val_mse: 4220048.0000 - val_mae: 1448.

2980/2980 [==============================] - 11s 4ms/step - loss: 4947908.0000 - mse: 4947908.0000 - mae: 1325.7705 - val_loss: 4139889.2500 - val_mse: 4139889.2500 - val_mae: 1418.2666
Epoch 541/1000
2980/2980 [==============================] - 13s 4ms/step - loss: 4583861.0000 - mse: 4583861.0000 - mae: 1324.7169 - val_loss: 4039661.2500 - val_mse: 4039661.2500 - val_mae: 1383.8029
Epoch 542/1000
2980/2980 [==============================] - 13s 4ms/step - loss: 4517862.5000 - mse: 4517862.5000 - mae: 1321.0509 - val_loss: 3991070.0000 - val_mse: 3991070.0000 - val_mae: 1343.8213
Epoch 543/1000
2980/2980 [==============================] - 11s 4ms/step - loss: 4496930.0000 - mse: 4496930.0000 - mae: 1325.3615 - val_loss: 3891917.2500 - val_mse: 3891917.2500 - val_mae: 1280.8574
Epoch 544/1000
2980/2980 [==============================] - 11s 4ms/step - loss: 5433784.5000 - mse: 5433784.5000 - mae: 1326.0466 - val_loss: 3950074.5000 - val_mse: 3950074.5000 - val_mae: 1316.8827
Epoch 545/

Epoch 581/1000
2980/2980 [==============================] - 11s 4ms/step - loss: 4627124.0000 - mse: 4627124.0000 - mae: 1334.3135 - val_loss: 4280718.0000 - val_mse: 4280718.0000 - val_mae: 1464.8741
Epoch 582/1000
2980/2980 [==============================] - 13s 4ms/step - loss: 4645943.5000 - mse: 4645943.5000 - mae: 1333.7085 - val_loss: 4102441.7500 - val_mse: 4102441.7500 - val_mae: 1374.3219
Epoch 583/1000
2980/2980 [==============================] - 13s 4ms/step - loss: 4552016.5000 - mse: 4552016.5000 - mae: 1329.2172 - val_loss: 4040302.2500 - val_mse: 4040302.2500 - val_mae: 1339.1097
Epoch 584/1000
2980/2980 [==============================] - 15s 5ms/step - loss: 4583628.0000 - mse: 4583628.0000 - mae: 1328.3794 - val_loss: 4021716.0000 - val_mse: 4021716.0000 - val_mae: 1330.5839
Epoch 585/1000
2980/2980 [==============================] - 15s 5ms/step - loss: 4494010.0000 - mse: 4494010.0000 - mae: 1326.1399 - val_loss: 4428358.0000 - val_mse: 4428358.0000 - val_mae: 1539.

2980/2980 [==============================] - 14s 5ms/step - loss: 6458594.5000 - mse: 6458594.5000 - mae: 1323.2151 - val_loss: 3886195.7500 - val_mse: 3886195.7500 - val_mae: 1254.5656
Epoch 622/1000
2980/2980 [==============================] - 14s 5ms/step - loss: 4603606.5000 - mse: 4603606.5000 - mae: 1313.6504 - val_loss: 4677739.5000 - val_mse: 4677739.5000 - val_mae: 1608.0145
Epoch 623/1000
2980/2980 [==============================] - 16s 5ms/step - loss: 8807993.0000 - mse: 8807993.0000 - mae: 1323.8756 - val_loss: 4069904.2500 - val_mse: 4069904.2500 - val_mae: 1351.8046
Epoch 624/1000
2980/2980 [==============================] - 16s 5ms/step - loss: 17669024.0000 - mse: 17669024.0000 - mae: 1330.7544 - val_loss: 3953177.5000 - val_mse: 3953177.5000 - val_mae: 1311.7113
Epoch 625/1000
2980/2980 [==============================] - 12s 4ms/step - loss: 4561507.5000 - mse: 4561507.5000 - mae: 1314.9951 - val_loss: 3851124.7500 - val_mse: 3851124.7500 - val_mae: 1256.8132
Epoch 62

Epoch 662/1000
2980/2980 [==============================] - 12s 4ms/step - loss: 4613921.5000 - mse: 4613921.5000 - mae: 1329.2777 - val_loss: 4193018.2500 - val_mse: 4193018.2500 - val_mae: 1458.3698
Epoch 663/1000
2980/2980 [==============================] - 16s 5ms/step - loss: 4648455.0000 - mse: 4648455.0000 - mae: 1327.6731 - val_loss: 3871645.7500 - val_mse: 3871645.7500 - val_mae: 1243.2279
Epoch 664/1000
2980/2980 [==============================] - 20s 7ms/step - loss: 5170155.0000 - mse: 5170155.0000 - mae: 1335.2061 - val_loss: 3996353.0000 - val_mse: 3996353.0000 - val_mae: 1267.3567
Epoch 665/1000
2980/2980 [==============================] - 18s 6ms/step - loss: 4640504.5000 - mse: 4640504.5000 - mae: 1327.8868 - val_loss: 4476454.5000 - val_mse: 4476454.5000 - val_mae: 1544.5721
Epoch 666/1000
2980/2980 [==============================] - 16s 5ms/step - loss: 4606574.5000 - mse: 4606574.5000 - mae: 1335.2343 - val_loss: 4010362.0000 - val_mse: 4010362.0000 - val_mae: 1255.

...

Epoch 826/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 5012123.0000 - mse: 5012123.0000 - mae: 1363.7809 - val_loss: 4334273.5000 - val_mse: 4334273.5000 - val_mae: 1429.2125
Epoch 827/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 4714799.0000 - mse: 4714799.0000 - mae: 1349.8583 - val_loss: 3948176.2500 - val_mse: 3948176.2500 - val_mae: 1281.0485
Epoch 828/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 4814222.0000 - mse: 4814222.0000 - mae: 1350.0994 - val_loss: 3921096.5000 - val_mse: 3921096.5000 - val_mae: 1276.9174
Epoch 829/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 5134284.0000 - mse: 5134284.0000 - mae: 1350.2758 - val_loss: 3916177.5000 - val_mse: 3916177.5000 - val_mae: 1288.5941
Epoch 830/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 4645159.5000 - mse: 4645159.5000 - mae: 1338.3710 - val_loss: 4653628.0000 - val_mse: 4653628.0000 - val_mae: 1618.2905


Epoch 867/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 4748210.5000 - mse: 4748210.5000 - mae: 1344.9003 - val_loss: 4254221.0000 - val_mse: 4254221.0000 - val_mae: 1411.0044
Epoch 868/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 4830927.0000 - mse: 4830927.0000 - mae: 1342.9485 - val_loss: 4028834.5000 - val_mse: 4028834.5000 - val_mae: 1259.8632
Epoch 869/1000
2980/2980 [==============================] - 10s 3ms/step - loss: 16416510.0000 - mse: 16416510.0000 - mae: 1356.8651 - val_loss: 3898985.5000 - val_mse: 3898985.5000 - val_mae: 1242.8169
Epoch 870/1000
2980/2980 [==============================] - 10s 3ms/step - loss: 8500716.0000 - mse: 8500716.0000 - mae: 1349.2167 - val_loss: 4071046.5000 - val_mse: 4071046.5000 - val_mae: 1343.1072
Epoch 871/1000
2980/2980 [==============================] - 10s 3ms/step - loss: 32902012.0000 - mse: 32902012.0000 - mae: 1375.0106 - val_loss: 4089975.7500 - val_mse: 4089975.7500 - val_mae: 140

Epoch 908/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 4627544.0000 - mse: 4627544.0000 - mae: 1337.9756 - val_loss: 3998932.5000 - val_mse: 3998932.5000 - val_mae: 1263.2977
Epoch 909/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 4594513.5000 - mse: 4594513.5000 - mae: 1330.8929 - val_loss: 3940031.2500 - val_mse: 3940031.2500 - val_mae: 1237.7933
Epoch 910/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 4579418.0000 - mse: 4579418.0000 - mae: 1331.5326 - val_loss: 4723382.5000 - val_mse: 4723382.5000 - val_mae: 1656.0756
Epoch 911/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 4567210.5000 - mse: 4567210.5000 - mae: 1330.1259 - val_loss: 3950648.2500 - val_mse: 3950648.2500 - val_mae: 1265.6962
Epoch 912/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 4526498.5000 - mse: 4526498.5000 - mae: 1329.1315 - val_loss: 4207354.0000 - val_mse: 4207354.0000 - val_mae: 1450.5717


Epoch 949/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 4663908.5000 - mse: 4663908.5000 - mae: 1340.3730 - val_loss: 4009025.5000 - val_mse: 4009025.5000 - val_mae: 1240.2805
Epoch 950/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 4667357.5000 - mse: 4667357.5000 - mae: 1341.6793 - val_loss: 3941848.5000 - val_mse: 3941848.5000 - val_mae: 1332.6191
Epoch 951/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 4714752.0000 - mse: 4714752.0000 - mae: 1344.2744 - val_loss: 4490756.5000 - val_mse: 4490756.5000 - val_mae: 1578.7639
Epoch 952/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 4714234.5000 - mse: 4714234.5000 - mae: 1345.5201 - val_loss: 4250088.0000 - val_mse: 4250088.0000 - val_mae: 1280.5409
Epoch 953/1000
2980/2980 [==============================] - 8s 3ms/step - loss: 4771162.5000 - mse: 4771162.5000 - mae: 1363.6226 - val_loss: 4039792.5000 - val_mse: 4039792.5000 - val_mae: 1267.4218


Epoch 990/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 4715220.0000 - mse: 4715220.0000 - mae: 1349.5559 - val_loss: 3991700.0000 - val_mse: 3991700.0000 - val_mae: 1309.2238
Epoch 991/1000
2980/2980 [==============================] - 8s 3ms/step - loss: 4754326.0000 - mse: 4754326.0000 - mae: 1350.8444 - val_loss: 3896304.5000 - val_mse: 3896304.5000 - val_mae: 1261.2064
Epoch 992/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 4719061.0000 - mse: 4719061.0000 - mae: 1349.7131 - val_loss: 4007879.0000 - val_mse: 4007879.0000 - val_mae: 1247.0884
Epoch 993/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 4663359.0000 - mse: 4663359.0000 - mae: 1346.9515 - val_loss: 3952970.5000 - val_mse: 3952970.5000 - val_mae: 1266.0128
Epoch 994/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 4668233.0000 - mse: 4668233.0000 - mae: 1342.3341 - val_loss: 4169489.0000 - val_mse: 4169489.0000 - val_mae: 1406.9438


In [ ]:
%%time
init_epochs = 1000
epochs_now = 5000
history = model.fit(X_train, y_train, epochs=epochs_now, validation_data=(X_val, y_val), initial_epoch=init_epochs)
mse_history += history.history['mse']
mae_history += history.history['mae']
model.save(f'relu_linear_6/model_{epochs_now}.h5')

Epoch 1001/5000
2980/2980 [==============================] - 9s 3ms/step - loss: 4736403.5000 - mse: 4736403.5000 - mae: 1350.2184 - val_loss: 4005961.2500 - val_mse: 4005961.2500 - val_mae: 1252.4774
Epoch 1002/5000
2980/2980 [==============================] - 9s 3ms/step - loss: 4738085.5000 - mse: 4738085.5000 - mae: 1347.6340 - val_loss: 4184522.7500 - val_mse: 4184522.7500 - val_mae: 1421.7451
Epoch 1003/5000
2980/2980 [==============================] - 9s 3ms/step - loss: 4748070.0000 - mse: 4748070.0000 - mae: 1352.8907 - val_loss: 4006188.2500 - val_mse: 4006188.2500 - val_mae: 1318.2599
Epoch 1004/5000
2980/2980 [==============================] - 9s 3ms/step - loss: 4741695.0000 - mse: 4741695.0000 - mae: 1357.7299 - val_loss: 4246840.5000 - val_mse: 4246840.5000 - val_mae: 1486.0878
Epoch 1005/5000
2980/2980 [==============================] - 10s 3ms/step - loss: 4692208.0000 - mse: 4692208.0000 - mae: 1340.6030 - val_loss: 4132233.2500 - val_mse: 4132233.2500 - val_mae: 1411

Epoch 1042/5000
2980/2980 [==============================] - 9s 3ms/step - loss: 4790104.0000 - mse: 4790104.0000 - mae: 1365.4912 - val_loss: 3954446.5000 - val_mse: 3954446.5000 - val_mae: 1260.3640
Epoch 1043/5000
2980/2980 [==============================] - 9s 3ms/step - loss: 4688698.5000 - mse: 4688698.5000 - mae: 1354.5946 - val_loss: 3925446.7500 - val_mse: 3925446.7500 - val_mae: 1253.2107
Epoch 1044/5000
2980/2980 [==============================] - 9s 3ms/step - loss: 4736118.5000 - mse: 4736118.5000 - mae: 1359.2422 - val_loss: 4031131.2500 - val_mse: 4031131.2500 - val_mae: 1260.1229
Epoch 1045/5000
2980/2980 [==============================] - 9s 3ms/step - loss: 4725230.5000 - mse: 4725230.5000 - mae: 1355.7117 - val_loss: 4002736.7500 - val_mse: 4002736.7500 - val_mae: 1252.0800
Epoch 1046/5000
2980/2980 [==============================] - 9s 3ms/step - loss: 4903732.0000 - mse: 4903732.0000 - mae: 1376.6914 - val_loss: 4011492.2500 - val_mse: 4011492.2500 - val_mae: 1312.

Epoch 1083/5000
2980/2980 [==============================] - 8s 3ms/step - loss: 4624591.0000 - mse: 4624591.0000 - mae: 1346.3151 - val_loss: 4131398.7500 - val_mse: 4131398.7500 - val_mae: 1255.4788
Epoch 1084/5000
2980/2980 [==============================] - 9s 3ms/step - loss: 4697955.5000 - mse: 4697955.5000 - mae: 1351.2911 - val_loss: 3958346.2500 - val_mse: 3958346.2500 - val_mae: 1270.1331
Epoch 1085/5000
2980/2980 [==============================] - 8s 3ms/step - loss: 4684721.0000 - mse: 4684721.0000 - mae: 1353.6121 - val_loss: 4111715.7500 - val_mse: 4111715.7500 - val_mae: 1309.0393
Epoch 1086/5000
2980/2980 [==============================] - 9s 3ms/step - loss: 4755317.5000 - mse: 4755317.5000 - mae: 1355.2173 - val_loss: 4049669.0000 - val_mse: 4049669.0000 - val_mae: 1250.0345
Epoch 1087/5000
2980/2980 [==============================] - 9s 3ms/step - loss: 4768338.0000 - mse: 4768338.0000 - mae: 1360.1796 - val_loss: 3974251.5000 - val_mse: 3974251.5000 - val_mae: 1316.

Epoch 1124/5000
2980/2980 [==============================] - 9s 3ms/step - loss: 4690483.5000 - mse: 4690483.5000 - mae: 1346.9816 - val_loss: 4539059.0000 - val_mse: 4539059.0000 - val_mae: 1588.7384
Epoch 1125/5000
2980/2980 [==============================] - 9s 3ms/step - loss: 4695180.5000 - mse: 4695180.5000 - mae: 1348.3892 - val_loss: 3926146.7500 - val_mse: 3926146.7500 - val_mae: 1309.3138
Epoch 1126/5000
2980/2980 [==============================] - 9s 3ms/step - loss: 4768921.0000 - mse: 4768921.0000 - mae: 1349.4391 - val_loss: 4042163.0000 - val_mse: 4042163.0000 - val_mae: 1325.6018
Epoch 1127/5000
2980/2980 [==============================] - 8s 3ms/step - loss: 4719097.0000 - mse: 4719097.0000 - mae: 1355.9889 - val_loss: 4022449.0000 - val_mse: 4022449.0000 - val_mae: 1313.4678
Epoch 1128/5000
2980/2980 [==============================] - 9s 3ms/step - loss: 4733389.5000 - mse: 4733389.5000 - mae: 1353.9419 - val_loss: 3971693.0000 - val_mse: 3971693.0000 - val_mae: 1277.

Epoch 1165/5000
2980/2980 [==============================] - 9s 3ms/step - loss: 4653053.5000 - mse: 4653053.5000 - mae: 1339.5170 - val_loss: 4021644.7500 - val_mse: 4021644.7500 - val_mae: 1239.4801
Epoch 1166/5000
2980/2980 [==============================] - 8s 3ms/step - loss: 4751660.5000 - mse: 4751660.5000 - mae: 1348.0012 - val_loss: 3976408.7500 - val_mse: 3976408.7500 - val_mae: 1237.1199
Epoch 1167/5000
2980/2980 [==============================] - 8s 3ms/step - loss: 4657983.5000 - mse: 4657983.5000 - mae: 1345.1530 - val_loss: 4181926.5000 - val_mse: 4181926.5000 - val_mae: 1407.7683
Epoch 1168/5000
2980/2980 [==============================] - 9s 3ms/step - loss: 4724230.0000 - mse: 4724230.0000 - mae: 1344.0952 - val_loss: 4099461.7500 - val_mse: 4099461.7500 - val_mae: 1347.8263
Epoch 1169/5000
2980/2980 [==============================] - 8s 3ms/step - loss: 4686928.0000 - mse: 4686928.0000 - mae: 1344.0511 - val_loss: 4150524.5000 - val_mse: 4150524.5000 - val_mae: 1271.

2980/2980 [==============================] - 34s 12ms/step - loss: 4586462.5000 - mse: 4586462.5000 - mae: 1333.7871 - val_loss: 4051766.2500 - val_mse: 4051766.2500 - val_mae: 1246.7875
Epoch 1206/5000
2980/2980 [==============================] - 33s 11ms/step - loss: 4633382.0000 - mse: 4633382.0000 - mae: 1341.6086 - val_loss: 3942499.2500 - val_mse: 3942499.2500 - val_mae: 1270.9481
Epoch 1207/5000
2980/2980 [==============================] - 33s 11ms/step - loss: 4684168.5000 - mse: 4684168.5000 - mae: 1345.6554 - val_loss: 4041349.7500 - val_mse: 4041349.7500 - val_mae: 1247.3644
Epoch 1208/5000
2980/2980 [==============================] - 32s 11ms/step - loss: 4638629.5000 - mse: 4638629.5000 - mae: 1340.7120 - val_loss: 4152591.7500 - val_mse: 4152591.7500 - val_mae: 1422.7594
Epoch 1209/5000
2980/2980 [==============================] - 33s 11ms/step - loss: 4568036.0000 - mse: 4568036.0000 - mae: 1332.4985 - val_loss: 4071444.5000 - val_mse: 4071444.5000 - val_mae: 1305.0624
E

2980/2980 [==============================] - 24s 8ms/step - loss: 4677612.5000 - mse: 4677612.5000 - mae: 1343.9091 - val_loss: 4003120.7500 - val_mse: 4003120.7500 - val_mae: 1313.7836
Epoch 1246/5000
2980/2980 [==============================] - 23s 8ms/step - loss: 4661724.0000 - mse: 4661724.0000 - mae: 1345.2723 - val_loss: 4393131.5000 - val_mse: 4393131.5000 - val_mae: 1515.3000
Epoch 1247/5000
2980/2980 [==============================] - 23s 8ms/step - loss: 4681269.5000 - mse: 4681269.5000 - mae: 1343.8341 - val_loss: 3904880.5000 - val_mse: 3904880.5000 - val_mae: 1242.2709
Epoch 1248/5000
2980/2980 [==============================] - 21s 7ms/step - loss: 4692128.0000 - mse: 4692128.0000 - mae: 1343.0769 - val_loss: 4006503.5000 - val_mse: 4006503.5000 - val_mae: 1247.0925
Epoch 1249/5000
2980/2980 [==============================] - 24s 8ms/step - loss: 4676587.0000 - mse: 4676587.0000 - mae: 1342.1932 - val_loss: 4172527.2500 - val_mse: 4172527.2500 - val_mae: 1246.5367
Epoch 

2980/2980 [==============================] - 21s 7ms/step - loss: 4640750.5000 - mse: 4640750.5000 - mae: 1337.3634 - val_loss: 4007800.0000 - val_mse: 4007800.0000 - val_mae: 1289.2072
Epoch 1286/5000
2980/2980 [==============================] - 24s 8ms/step - loss: 4647378.0000 - mse: 4647378.0000 - mae: 1340.5422 - val_loss: 3995334.2500 - val_mse: 3995334.2500 - val_mae: 1261.5688
Epoch 1287/5000
2980/2980 [==============================] - 23s 8ms/step - loss: 4784879.5000 - mse: 4784879.5000 - mae: 1342.5312 - val_loss: 4172269.7500 - val_mse: 4172269.7500 - val_mae: 1400.9337
Epoch 1288/5000
2980/2980 [==============================] - 22s 7ms/step - loss: 4661192.5000 - mse: 4661192.5000 - mae: 1338.7299 - val_loss: 4066047.7500 - val_mse: 4066047.7500 - val_mae: 1362.6630
Epoch 1289/5000
2980/2980 [==============================] - 22s 8ms/step - loss: 4691344.5000 - mse: 4691344.5000 - mae: 1341.2823 - val_loss: 4180981.5000 - val_mse: 4180981.5000 - val_mae: 1397.8647
Epoch 

In [ ]:
plt.plot(mae_history)
plt.show()
plt.plot(mse_history)
plt.show()